<a href="https://colab.research.google.com/github/Pearlkakande/machinelearning/blob/main/models/models4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


MODEL TRAINING FOR GRAPH-BASED KNOWLEDGE DISTILLATION AND FEATURE LEARNING FOR BOOK RECOMMENDATIONS

Ten models are to be trained with model evaluation being handled by wandb

dataset= Eitanli/goodreads for the hugging face dataset library

SET UP

packages, dataset, wandb

In [2]:
# Install necessary packages (uncomment if needed)
!pip install datasets torch wandb sentence-transformers scikit-learn
!pip install torch-scatter torch-sparse torch-cluster torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.nn import GCNConv, GATConv  # and other layers as needed
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import wandb
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1E

WANDB

In [ ]:
# (Global W&B login is fine to do once)
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pearlkakande (pearlkakande-makerere-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Dataset Loading

In [4]:
# Load the dataset from Hugging Face
dataset = load_dataset("Eitanli/goodreads")["train"]  # assume 'train' split
df = pd.DataFrame(dataset)
print(df.columns.tolist())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/737 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


goodreads_data.csv:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

['Unnamed: 0', 'Book', 'Author', 'Description', 'Genres', 'Avg_Rating', 'Num_Ratings', 'URL']


GRAPH CONSTUCTION

Description embedding, spliting of dataset

the book, author and genre are nodes

edges show relationship and addding rating-based popularity as a node attribute or as edge weight

In [5]:
# Use a pre-trained sentence transformer to embed the book descriptions.
model_st = SentenceTransformer('all-MiniLM-L6-v2')
df['desc_emb'] = df['Description'].fillna("").apply(lambda x: model_st.encode(x))


# Split into train/test (e.g., 80/20 split by index for simplicity)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Build a simple heterogeneous graph (using PyG's HeteroData):
def build_hetero_graph(df):
    data = HeteroData()

    # Create book nodes: use description embeddings as features.
    book_emb = np.stack(df['desc_emb'].values)
    data['book'].x = torch.tensor(book_emb, dtype=torch.float)

    # For authors and genres, create a mapping.
    authors = list(df['Author'].unique())
    genres = list(df['Genres'].unique())
    author2id = {a: i for i, a in enumerate(authors)}
    genre2id = {g: i for i, g in enumerate(genres)}

    # Create author nodes with one-hot features.
    data['author'].num_nodes = len(authors)
    data['author'].x = F.one_hot(torch.arange(len(authors)), num_classes=len(authors)).float()

    # Create genre nodes.
    data['genre'].num_nodes = len(genres)
    data['genre'].x = F.one_hot(torch.arange(len(genres)), num_classes=len(genres)).float()

    # Build edges: book -> author and book -> genre.
    book_ids = np.arange(len(df))
    author_ids = [author2id[a] for a in df['Author']]
    genre_ids = [genre2id[g] for g in df['Genres']]

    data['book', 'written_by', 'author'].edge_index = torch.tensor([book_ids, author_ids], dtype=torch.long)
    data['book', 'has_genre', 'genre'].edge_index = torch.tensor([book_ids, genre_ids], dtype=torch.long)

    # (Optional) Build book-to-book similarity edges based on cosine similarity between description embeddings:
    # For simplicity, here we add edges for pairs with similarity > 0.9.
    from sklearn.metrics.pairwise import cosine_similarity
    sim_matrix = cosine_similarity(book_emb)
    src, dst = np.where(sim_matrix > 0.9)
    # remove self-loops:
    mask = src != dst
    data['book', 'similar_to', 'book'].edge_index = torch.tensor([src[mask], dst[mask]], dtype=torch.long)

    # You can also include rating-based popularity as a node attribute or as edge weight.
    # For example, store ratings_count and average_rating in a separate tensor.
    #convert num ratings to int type for later manipulation
    df['Num_Ratings'] = df['Num_Ratings'].str.replace(',', '').astype(int)
    data['book'].ratings_count = torch.tensor(df['Num_Ratings'].values.astype(float), dtype=torch.float)
    data['book'].average_rating = torch.tensor(df['Avg_Rating'].values.astype(float), dtype=torch.float)

    return data

data = build_hetero_graph(train_df)  # build graph from training data

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-5-209a83e14140>:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  data['book', 'written_by', 'author'].edge_index = torch.tensor([book_ids, author_ids], dtype=torch.long)


MODEL 1 LightGCN for Book Recommendation



In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import wandb

# Initialize wandb before training the model
wandb.init(project="book-recommendation",    # Replace with your project name
           entity="pearlkakande-makerere-university",       # Replace with your wandb username or team name
           config={"model": "LightGCN", "epochs": 30, "learning_rate": 0.01})

# Assume wandb.login() was called in an earlier cell

class LightGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=2):
        super(LightGCN, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        # Final linear layer: note we later concatenate popularity signal so input dims change.
        # Here we assume hidden_channels remains unchanged; adjust if needed.
        self.lin = nn.Linear(hidden_channels + 1, 2)  # binary classification: warm vs cold

    def forward(self, data):
        # Use only 'book' nodes for prediction.
        x = data['book'].x  # initial features from description embeddings
        for conv in self.convs:
            x = conv(x, data['book', 'similar_to', 'book'].edge_index)
            x = F.relu(x)
        # Concatenate popularity signal (ratings_count)
        ratings = data['book'].ratings_count.unsqueeze(1)
        x_cat = torch.cat([x, ratings], dim=1)
        out = self.lin(x_cat)
        return out, x_cat  # also return latent embedding for potential distillation

# Define warm items based on ratings_count > threshold.
threshold = train_df['Num_Ratings'].median()
labels = (train_df['Num_Ratings'] > threshold).astype(int).values
labels = torch.tensor(labels, dtype=torch.long)

# Set device and create model instance
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = LightGCN(in_channels=data['book'].x.size(1), hidden_channels=64, num_layers=2).to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.01)

# Move data to device
data['book'].x = data['book'].x.to(device)
data['book', 'similar_to', 'book'].edge_index = data['book', 'similar_to', 'book'].edge_index.to(device)
data['book'].ratings_count = data['book'].ratings_count.to(device)
labels = labels.to(device)

# Training loop for 30 epochs
num_epochs = 30
model1.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    out, _ = model1(data)
    loss = F.cross_entropy(out, labels)
    loss.backward()
    optimizer.step()

    # Evaluate on training data (for demo purposes)
    preds = out.argmax(dim=1).cpu().numpy()
    truelabels = labels.cpu().numpy()
    acc = accuracy_score(truelabels, preds)
    prec = precision_score(truelabels, preds, zero_division=0)
    rec = recall_score(truelabels, preds, zero_division=0)
    f1 = f1_score(truelabels, preds, zero_division=0)

    wandb.log({
        "Model": "LightGCN",
        "epoch": epoch,
        "loss": loss.item(),
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    })

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss={loss.item():.4f}, Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")

# After training, create a summary table for the training set.
results = {"Model": "LightGCN", "Loss": loss.item(), "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}
print(pd.DataFrame([results]))

# Finish the wandb run
wandb.finish()


Epoch 0: Loss=50.7437, Acc=0.5009, Prec=0.5004, Rec=1.0000, F1=0.6671
Epoch 10: Loss=73.9310, Acc=0.6894, Prec=0.6168, Rec=1.0000, F1=0.7630
Epoch 20: Loss=72.4205, Acc=0.7999, Prec=0.7142, Rec=1.0000, F1=0.8332
      Model       Loss  Accuracy  Precision  Recall        F1
0  LightGCN  13.945383  0.923125   0.866739     1.0  0.928613


accuracy,▁▃▁▄▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
f1,▆▆▁▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
loss,▁▁█▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
precision,▅▅▁▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███
recall,██▁███████████████████████████
Model,LightGCN
accuracy,0.92312
epoch,29
f1,0.92861
loss,13.94538


MODEL 2 Graph Attention Network (GAT) for Books


In [11]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import wandb

# Initialize wandb before training the model
wandb.init(project="book-recommendation",    # Replace with your project name
           entity="pearlkakande-makerere-university",     # Replace with your wandb username or team name
           config={"model": "GATBook", "epochs": 30, "learning_rate": 0.005})

class GATBook(nn.Module):
    def __init__(self, in_channels, hidden_channels, heads=2, num_layers=2):
        super(GATBook, self).__init__()
        self.gat_layers = nn.ModuleList()
        self.gat_layers.append(GATConv(in_channels, hidden_channels, heads=heads))
        for _ in range(num_layers - 1):
            self.gat_layers.append(GATConv(hidden_channels * heads, hidden_channels, heads=heads))
        self.lin = nn.Linear(hidden_channels * heads + 1, 2)  # add ratings_count

    def forward(self, data):
        x = data['book'].x
        for gat in self.gat_layers:
            x = gat(x, data['book', 'similar_to', 'book'].edge_index)
            x = F.elu(x)
        ratings = data['book'].ratings_count.unsqueeze(1)
        x = torch.cat([x, ratings], dim=1)
        out = self.lin(x)
        return out

# Define warm items based on ratings_count > threshold.
threshold = train_df['Num_Ratings'].median()
labels = (train_df['Num_Ratings'] > threshold).astype(int).values
labels = torch.tensor(labels, dtype=torch.long)

# Set device and create model instance
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = GATBook(in_channels=data['book'].x.size(1), hidden_channels=32, heads=2).to(device)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.005)

# Move data to device
data['book'].x = data['book'].x.to(device)
data['book', 'similar_to', 'book'].edge_index = data['book', 'similar_to', 'book'].edge_index.to(device)
data['book'].ratings_count = data['book'].ratings_count.to(device)
labels = labels.to(device)

# Training loop for 30 epochs
num_epochs = 30
model2.train()
for epoch in range(num_epochs):
    optimizer2.zero_grad()
    out = model2(data)
    loss = F.cross_entropy(out, labels)
    loss.backward()
    optimizer2.step()

    # Evaluate on training data (for demo purposes)
    preds = out.argmax(dim=1).cpu().numpy()
    truelabels = labels.cpu().numpy()
    acc = accuracy_score(truelabels, preds)
    prec = precision_score(truelabels, preds, zero_division=0)
    rec = recall_score(truelabels, preds, zero_division=0)
    f1 = f1_score(truelabels, preds, zero_division=0)

    wandb.log({
        "Model": "GATBook",
        "epoch": epoch,
        "loss": loss.item(),
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    })

    if epoch % 10 == 0:
        print(f"GAT Epoch {epoch}: Loss={loss.item():.4f}, Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")

# After training, create a summary table for the training set.
results2 = {"Model": "GATBook", "Loss": loss.item(), "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}
print(pd.DataFrame([results2]))

# Finish the wandb run
wandb.finish()


GAT Epoch 0: Loss=212.3099, Acc=0.5000, Prec=0.5000, Rec=1.0000, F1=0.6667
GAT Epoch 10: Loss=47.3403, Acc=0.7136, Prec=0.6358, Rec=1.0000, F1=0.7774
GAT Epoch 20: Loss=30.4098, Acc=0.7640, Prec=0.6793, Rec=1.0000, F1=0.8091
     Model       Loss  Accuracy  Precision  Recall        F1
0  GATBook  59.578922  0.766125   0.681315     1.0  0.810455


accuracy,▁▁▁▂▂▂▃▃▃▄▄▅▅█▁▁▇▅▅▅▅▅▅▅▅▅▅▅▅▅
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
f1,▆▆▆▆▆▆▆▆▆▆▇▇▇█▁▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇
loss,▃▃▃▃▂▂▂▂▂▂▁▁▁▁█▄▁▁▁▁▁▁▁▁▁▁▂▂▂▂
precision,▅▅▅▅▅▅▅▅▅▅▆▆▆█▁▁▇▆▆▆▆▆▆▆▆▆▆▆▆▆
recall,██████████████▁▁██████████████
Model,GATBook
accuracy,0.76612
epoch,29
f1,0.81045
loss,59.57892


MODEL 3

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import wandb

# Initialize wandb before training the model
wandb.init(project="book-recommendation",    # Replace with your project name
           entity="pearlkakande-makerere-university",       # your wandb username or team name
           config={"model": "PGD", "epochs": 30, "learning_rate": 0.01})

# Define Teacher and Student models for PGD
class TeacherGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(TeacherGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, data):
        x = data['book'].x
        x = F.relu(self.conv1(x, data['book', 'similar_to', 'book'].edge_index))
        x = self.conv2(x, data['book', 'similar_to', 'book'].edge_index)
        return x

class StudentGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(StudentGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels + 1, 2)  # +1 for ratings_count feature

    def forward(self, data):
        x = data['book'].x
        x = F.relu(self.conv1(x, data['book', 'similar_to', 'book'].edge_index))
        x = self.conv2(x, data['book', 'similar_to', 'book'].edge_index)
        ratings = data['book'].ratings_count.unsqueeze(1)
        x_cat = torch.cat([x, ratings], dim=1)
        out = self.lin(x_cat)
        return out, x

# Define warm items based on ratings_count > threshold.
# Here we use 'ratings_count' from train_df and 'average_rating' for the teacher's target.
threshold = train_df['Num_Ratings'].median()
labels = (train_df['Num_Ratings'] > threshold).astype(int).values
labels = torch.tensor(labels, dtype=torch.long)

# Set device and create model instances
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
teacher = TeacherGCN(in_channels=data['book'].x.size(1), hidden_channels=64).to(device)
student = StudentGCN(in_channels=data['book'].x.size(1), hidden_channels=64).to(device)

optimizer_teacher = torch.optim.Adam(teacher.parameters(), lr=0.01)
optimizer_student = torch.optim.Adam(student.parameters(), lr=0.01)

# Move data and labels to device
data['book'].x = data['book'].x.to(device)
data['book', 'similar_to', 'book'].edge_index = data['book', 'similar_to', 'book'].edge_index.to(device)
data['book'].ratings_count = data['book'].ratings_count.to(device)
data['book'].average_rating = data['book'].average_rating.to(device)  # used as target for teacher
labels = labels.to(device)

num_epochs = 30

# --- Teacher Training ---
teacher.train()
for epoch in range(num_epochs):
    optimizer_teacher.zero_grad()
    teacher_emb = teacher(data)
    # Use a regression loss to predict average_rating as a proxy for CF signals.
    target = data['book'].average_rating.unsqueeze(1)
    # Here we take the mean over the embedding dimensions as a simple proxy.
    loss_teacher = F.mse_loss(teacher_emb.mean(dim=1, keepdim=True), target)
    loss_teacher.backward()
    optimizer_teacher.step()
    wandb.log({"Model": "PGD_Teacher", "epoch": epoch, "loss": loss_teacher.item()})
    if epoch % 10 == 0:
        print(f"Teacher Epoch {epoch}: Loss={loss_teacher.item():.4f}")

# Freeze teacher parameters
for param in teacher.parameters():
    param.requires_grad = False

# --- Student Training with Knowledge Distillation ---
student.train()
alpha = 0.7  # weight for distillation loss; 0.3 for ground truth classification
for epoch in range(num_epochs):
    optimizer_student.zero_grad()
    out_student, student_emb = student(data)
    with torch.no_grad():
        teacher_emb = teacher(data)
    # Distillation loss: MSE between student and teacher embeddings.
    loss_distill = F.mse_loss(student_emb, teacher_emb)
    # Classification loss: cross-entropy loss against warm/cold labels.
    loss_class = F.cross_entropy(out_student, labels)
    loss_total = alpha * loss_distill + (1 - alpha) * loss_class
    loss_total.backward()
    optimizer_student.step()

    preds = out_student.argmax(dim=1).cpu().numpy()
    truelabels = labels.cpu().numpy()
    acc = accuracy_score(truelabels, preds)
    prec = precision_score(truelabels, preds, zero_division=0)
    rec = recall_score(truelabels, preds, zero_division=0)
    f1 = f1_score(truelabels, preds, zero_division=0)

    wandb.log({"Model": "PGD_Student", "epoch": epoch, "loss_total": loss_total.item(),
               "distill_loss": loss_distill.item(), "class_loss": loss_class.item(),
               "accuracy": acc, "precision": prec, "recall": rec, "f1": f1})
    if epoch % 10 == 0:
        print(f"Student Epoch {epoch}: Total Loss={loss_total.item():.4f}")

results3 = {"Model": "PGD (Student)", "Loss": loss_total.item(), "Accuracy": acc,
            "Precision": prec, "Recall": rec, "F1": f1}
print(pd.DataFrame([results3]))

# Finish the wandb run
wandb.finish()


Teacher Epoch 0: Loss=16.7376
Teacher Epoch 10: Loss=2.3144
Teacher Epoch 20: Loss=1.3358
Student Epoch 0: Total Loss=4575.2148
Student Epoch 10: Total Loss=18.6114
Student Epoch 20: Total Loss=63.5808
           Model       Loss  Accuracy  Precision  Recall        F1
0  PGD (Student)  75.982819  0.723625   0.644019     1.0  0.783469


accuracy,▅▅▅▄▄▃▃▂▁▅▅▅▅▅▅▅▅▅▅▆▇▇▇▇██████
class_loss,█▇▆▆▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
distill_loss,██▇▇▆▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▅▆▇▇▇▇█▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██
f1,▁▁▁▁▁▁▁▁▁▇▇▇▇▇▇▇▇▇▇▇██████████
loss,██▇▇▆▆▅▄▃▂▂▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
loss_total,█▇▆▆▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▁▁▁▁▁▁▁▁▆▆▆▆▆▆▆▆▆▆▇▇▇████████
recall,▁▁▁▁▁▁▁▁▁█████████████████████
Model,PGD_Student
accuracy,0.72362


MODEL 4 Hybrid Graph-based Model for Cold-Start Recommendations

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import wandb

# Initialize wandb before training the model
wandb.init(project="book-recommendation",    # Replace with your project name
           entity="pearlkakande-makerere-university",       # wandb username or team name
           config={"model": "Hybrid", "epochs": 30, "learning_rate": 0.005})

class HybridModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, text_dim):
        super(HybridModel, self).__init__()
        # Graph branch
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        # Text branch: assume text_dim is the dimension of description embedding
        self.fc_text = nn.Linear(text_dim, hidden_channels)
        # Fusion and final classifier (also add popularity)
        self.fc_final = nn.Linear(hidden_channels * 2 + 1, 2)

    def forward(self, data):
        # Graph branch on book nodes
        x_graph = F.relu(self.gcn1(data['book'].x, data['book', 'similar_to', 'book'].edge_index))
        x_graph = self.gcn2(x_graph, data['book', 'similar_to', 'book'].edge_index)
        # Text branch: here we assume data['book'].x already comes from description embeddings
        x_text = F.relu(self.fc_text(data['book'].x))
        # Combine graph and text features and add popularity signal
        ratings = data['book'].ratings_count.unsqueeze(1)
        x_comb = torch.cat([x_graph, x_text, ratings], dim=1)
        out = self.fc_final(x_comb)
        return out

# Define warm items based on ratings_count > threshold.
threshold = train_df['Num_Ratings'].median()
labels = (train_df['Num_Ratings'] > threshold).astype(int).values
labels = torch.tensor(labels, dtype=torch.long)

# Set device and create model instance
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model4 = HybridModel(in_channels=data['book'].x.size(1), hidden_channels=32, text_dim=data['book'].x.size(1)).to(device)
optimizer4 = torch.optim.Adam(model4.parameters(), lr=0.005)

# Move data to device
data['book'].x = data['book'].x.to(device)
data['book', 'similar_to', 'book'].edge_index = data['book', 'similar_to', 'book'].edge_index.to(device)
data['book'].ratings_count = data['book'].ratings_count.to(device)
labels = labels.to(device)

# Training loop for 30 epochs
num_epochs = 30
model4.train()
for epoch in range(num_epochs):
    optimizer4.zero_grad()
    out = model4(data)
    loss = F.cross_entropy(out, labels)
    loss.backward()
    optimizer4.step()

    # Evaluate on training data (for demo purposes)
    preds = out.argmax(dim=1).cpu().numpy()
    truelabels = labels.cpu().numpy()
    acc = accuracy_score(truelabels, preds)
    prec = precision_score(truelabels, preds, zero_division=0)
    rec = recall_score(truelabels, preds, zero_division=0)
    f1 = f1_score(truelabels, preds, zero_division=0)

    wandb.log({
        "Model": "Hybrid",
        "epoch": epoch,
        "loss": loss.item(),
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    })

    if epoch % 10 == 0:
        print(f"Hybrid Epoch {epoch}: Loss={loss.item():.4f}")

results4 = {"Model": "Hybrid", "Loss": loss.item(), "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}
print(pd.DataFrame([results4]))

# Finish the wandb run
wandb.finish()


Hybrid Epoch 0: Loss=4101.6519
Hybrid Epoch 10: Loss=62.0347
Hybrid Epoch 20: Loss=107.8661
    Model        Loss  Accuracy  Precision  Recall        F1
0  Hybrid  120.316452    0.5075   0.503778     1.0  0.670017


accuracy,▇▆▅▄▁█████████████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
f1,▁▁▁▁▁█████████████████████████
loss,█▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▁▁▁▁█████████████████████████
recall,▁▁▁▁▁█████████████████████████
Model,Hybrid
accuracy,0.5075
epoch,29
f1,0.67002
loss,120.31645


MODEL 5